In [1]:
from utilities.helpers import *
from utilities.Data_preprocessing_global import *
from implementations import *
from utilities.Hyperparameters_Logreg import *

### Importing the data

In [3]:
# You have to change the path for it to work
#data_path = r'C:\Users\natha\Documents\EPFL\Cours_MA1\ML\ML_course\projects\project1 - withGit\data\dataset'
data_path = "D:\\EPFL\\MA1\\Machine Learning\\Projet 1\\dataset"

In [4]:
x_train, x_test, y_train, train_ids, test_ids, headers_train = load_csv_data(data_path, sub_sample=False)

test


### Pre-processing the data

In [5]:
x_tr, x_val, y_tr, y_val, x_train_full, x_test_formatted, remaining_headers = data_preprocess(x_train, y_train, x_test, headers_train, model_labels = {0, 1}, ratio_miss = 0.1, ratio_train = 1, standardization = True)

See the different shapes : x_train (328135, 321), x_test (109379, 321), y_train (328135,), headers_train: 321
After preprocessing (train) : column with missing values {}, are there NaN ? False
After preprocessing (test) : column with missing values {}, are there NaN ? False
See the different shapes : x_tr (328135, 169), x_val (0, 169), y_tr (328135,), y_te(0,), x_test_formatted(109379, 169)


We intentionally select a split training vs validation of 100% vs 0% because this split will be done inside the K-Fold cross-validation.

### Hyperparameter tuning 

In [10]:
## Here we are defining 3 hyperparameters, the missing ratio of data acceptable in each column, the regularization term, the learning rate and the limit
## to discriminate the predictions to -1 or 1

# Here I only have a small list of parameter to verify that it all works correctly, feel free to adapt them
k_fold = 3
regularization_term = [0.01, 0.001]
learning_rate = [0.1, 0.01, 0.001]
limits = np.linspace(0.2,0.5,4)

initial_w = np.zeros(x_tr.shape[1])
max_iters = 1000

In [11]:
results = Hyperparameters(x_tr, y_tr, k_fold, max_iters, regularization_term, learning_rate, limits, initial_w)

In [ ]:
def Get_best_results(results):
    # Initialize variables to track the best F1-score and corresponding parameters
    best_f1_score = 0
    best_params = None

    # Loop through the results to find the best F1-score on the validation set
    for result in results:
        f1_score_val = result["av_f1_score"]

        # Update if a new best F1-score is found
        if f1_score_val > best_f1_score:
            best_f1_score = f1_score_val
            best_params = {
                "regularization_term": result["regularization_term"],
                "learning_rate": result["learning_rate"],
                "limit": result["limit"],
            }

    # Output the best F1-score and corresponding parameters
    print("Best F1-Score:", best_f1_score)
    print("Best Parameters and Metrics:", best_params)
    return best_f1_score, best_params

In [13]:
best_f1_score, best_params = Get_best_results(results)

Best F1-Score: 0.1764537250361379
Best Parameters and Metrics: {'regularization_term': 0.001, 'learning_rate': 0.1, 'limit': np.float64(0.2)}


### Submission

Now that we have found the best hyperparameters, we can use them to train our model one last time, on the whole dataset:

In [ ]:
lambda_ = 0.001
initial_w = np.zeros(x_tr.shape[1])
max_iters = 1000
gamma = 0.1

w_opt, loss_opt = reg_logistic_regression(y_tr, x_tr, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=0.6931471805599446
Current iteration=100, loss=0.23226847566932823
Current iteration=200, loss=0.22838252971242326
Current iteration=300, loss=0.2268618563995702
Current iteration=400, loss=0.2259811869250319
Current iteration=500, loss=0.22538842717410226
Current iteration=600, loss=0.22495506215754574
Current iteration=700, loss=0.22462094544609915
Current iteration=800, loss=0.22435362956031
Current iteration=900, loss=0.2241339002152805
loss=0.22394958817311608


In [17]:
y_pred, accuracy, precision, recall, f1_score = evaluate_performance(x_tr, y_tr, w_opt, {0,1}, 0.2)
print(accuracy)
print(f1_score)

0.9145016532829475
0.17555614328954713


In [15]:
best_limit = 0.2
best_w = w_opt
name = 'best_log_reg.csv'
model_labels = {0, 1}

y_pred_test = submission(x_test_formatted, test_ids, best_limit, best_w, name, model_labels)